In [ ]:
# this will make the plots appear directly inside the document
%matplotlib inline

In [ ]:
# libraries we need to perform the analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os, re
from scipy.optimize import curve_fit
from scipy import stats

# 2019-07-09 SPT data analysis compendium

The purpose of this document is to illustrate thoroughly the different steps of the data analysis of Single Particle Tracking (SPT) experiments.

**PREMISE**: This tutorial assumes basic knowledge of Python.
___

## Table of contents

1. **Loading the data**: How to load the data and put it into convenient data structures, taking into account the different requirements of the data analysis.

2. **Angle analysis**: we describe how to perform the analysis of the angles of the trajectories.

3. **Diffusion coefficient modeling**: we describe how to model diffusion coefficients.

----

## 1. Loading the data

We will look at directories that contain SPT trajectories. The directories will contain sub-directories named such as `Stack016_Cell4`. Inside those directories we will look for different files:

1. `Links in tracks statistics.csv` : a CSV-formatted file containing SPT information
2. `Spots in tracks statistics.txt` : a TSV-formatted file containing SPT information
3. `Spots in tracks statistics_MAP.txt` : a TSV-formatted file containing information on the tracks before the SPT can begin
4. `Track statistics.csv` : a CSV-formatted file with information on the quality of the tracks.

### 1.1 Loading a single SPT file
Let's start by an example. We will write a piece of code that will load the data of a single SPT experiment and put it into a variable that we will call `spt`.

In [ ]:
# for convenience, we will define a variable that we will call `spt_rootdir`, which is the folder
# that contains all the data that we will analyze
spt_rootdir = '/home/rcortini/work/CRG/projects/catadata/data'

# the file name that we will open
mydir = '%s/2_DMSO_R5020_Control/Stack009_Cell3'%(spt_rootdir)
spt_fname = '%s/Spots in tracks statistics.txt'%(mydir)

print(spt_fname)

In [ ]:
# now we use the `Pandas` Python package to parse the file. We call the `read_csv` function
# with the option `sep='\t'`, which means that the character that separates the fields is a tab
# character
spt = pd.read_csv(spt_fname, sep='\t')

Now we have the data of the experiment loaded into the `spt` variable. Let's have a look at how this variable looks like.

In [ ]:
spt.head()

In [ ]:
type(spt)


This variable is a Pandas `DataFrame`. The first thing to realize is that we will be interested in the single trajectories, which are identified by having the same `TRACK_ID`. To analyze the trajectories, therefore, we will use the function `groupby` from Pandas.

In [ ]:
spt_by_track = spt.groupby('TRACK_ID')

In [ ]:
spt_by_track.get_group(1)

The `spt_by_track` object is difficult to operate with directly. Most of the times we will be interested in the (x, y) coordinates of the trajectories. We can extract this information from this object by performing an iteration over the object. In the following piece of code we will do several things:

- initialize an empty list named `trajectories`
- unpack the `spt_by_track` structure by iterating over it
- extract the `x` and `y` coordinates of the tracks
- inserting the track (x, y) coordinates in our list

In [ ]:
# init empty list
trajectories = []

# iterate over the tracks. The `track_id` variable is the TRACK_ID field of the data frame.
for track_id, track in spt_by_track :
    
    # extract x and y from the trajectory
    x = track['POSITION_X']
    y = track['POSITION_Y']
    
    # here we define the trajectory. We put x and y in a list [x, y], then
    # we feed this to the function `np.array` which is numpy's way of initializing
    # an array, and then we transpose the array using `.T`. The transpose will make
    # our life easier, so we will have an array with N rows and 2 columns, where N is
    # the number of steps in the trajectory
    this_trajectory = np.array([x, y]).T
    
    # finally, append the current trajectory to the list of trajectories
    trajectories.append(this_trajectory)

Let's look at some basic information on the trajectories. How many trajectories do we have? We can use the function `len` which will give us the length of the list.

In [ ]:
len(trajectories)

If we take the first element of the `trajectories` list, we'll get this:

In [ ]:
trajectories[0]

Which are the (x, y) coordinates of a particle as a function of time. Notice that printing this variable shows `array` at the beginning, which is to say that what we're looking at is a numpy array. Numpy arrays have an important property which is the `shape`, which tells us the dimensions of the array.

In [ ]:
trajectories[0].shape

This particular trajectory contains 24 points.

We can plot this trajectory:

In [ ]:
# notice that the `plot` function takes at least two arguments: x and y. We access the x and y
# fields of our array by using [:, 0] and [:, 1] respectively. The `:` character means that we
# want all the points, whereas the 0 or 1 means we want the column number 0 or 1.
plt.plot(trajectories[0][:, 0], trajectories[0][:, 1])
plt.show()

We can also create a list that contains the number of points in each trajectory in the following way:

In [ ]:
# this way of creating a list is called "list comprehension" and is a very useful
# syntax feature of Python. It allows to initialize and create a list in one line of code.
trajectory_length = [t.shape[0] for t in trajectories]

We can now plot a histogram of the trajectory lengths.

In [ ]:
plt.hist(trajectory_length, bins=50)
plt.xlabel("Number of points in track")
plt.ylabel("Counts")
plt.show()

### 1.2 Quality control
We may want to avoid loading the tracks that do not reach a minimum quality. To do this, we will look at the information stored into the `Track statistics.csv` file.

In [ ]:
spt.head()

In [ ]:
# let's parse the track statistics file. Notice that we do not set the `sep` parameter
# as we did before. In fact, this is a CSV file and Pandas will parse it well by default.
track_statistics = pd.read_csv('%s/Track statistics.csv'%(mydir))

# let's have a look at what it looks like
track_statistics[['Label','TRACK_ID','TRACK_MEAN_QUALITY']].head()

One of the columns of this file is named `TRACK_ID`, and has the same meaning as the one stored in `spt`.

Here, we are interested in the parameter `TRACK_MEAN_QUALITY`. We aim at selecting the tracks that have a track mean quality that is over a certain threshold.

In [ ]:
# first, get the a list of the tracks that pass the quality control criterion
idx_tracks_to_keep = track_statistics.TRACK_MEAN_QUALITY > 50

# then, select the TRACK_ID of the tracks in this list
tracks_to_keep = track_statistics[idx_tracks_to_keep].TRACK_ID

Now that we have this, we can get the trajectories as we did before, but adding the piece of code that will exclude the tracks that are not in the list.

In [ ]:
# init empty list
trajectories = []

# iterate over the tracks. The `track_id` variable is the TRACK_ID field of the data frame.
for track_id, track in spt_by_track :
    
    # skip tracks that did not have sufficiently high average quality
    if track_id not in tracks_to_keep :
        continue
    
    # extract x and y from the trajectory
    x = track['POSITION_X']
    y = track['POSITION_Y']
    
    # here we define the trajectory. We put x and y in a list [x, y], then
    # we feed this to the function `np.array` which is numpy's way of initializing
    # an array, and then we transpose the array using `.T`. The transpose will make
    # our life easier, so we will have an array with N rows and 2 columns, where N is
    # the number of steps in the trajectory
    this_trajectory = np.array([x, y]).T
    
    # finally, append the current trajectory to the list of trajectories
    trajectories.append(this_trajectory)
    
print("Loaded %d tracks"%(len(trajectories)))

As we can see here, many tracks were thrown away because of the quality criterion.

## 1.3 A function to load a track
Now we can encapsulate the code that we wrote in a single function that will load the data, perform the quality control, and return the list of trajectories that we want.

In [ ]:
track_statistics

In [ ]:
def load_spt(datadir, quality = None) :
    # load the SPT data file
    spt_fname = '%s/Spots in tracks statistics.txt'%(datadir)
    spt = pd.read_csv(spt_fname, sep='\t')
    
    # group by TRACK_ID
    spt_by_track = spt.groupby('TRACK_ID')

    # get quality of tracks and list of excluded tracks
    if quality is not None :
        track_statistics = pd.read_csv('%s/Track statistics.csv'%(datadir))
        idx_tracks_to_exclude = track_statistics.TRACK_MEAN_QUALITY < quality
        tracks_to_exclude = track_statistics[idx_tracks_to_exclude].TRACK_ID
    else :
        tracks_to_exclude = []
    
    # extract trajectories
    trajectories = []
    good_track_ids = []
    for track_id, track in spt_by_track :
        
        # skip tracks that did not have sufficiently high average quality
        if track_id in tracks_to_exclude.values :
            continue

        # extract x and y from the trajectory
        x = track['POSITION_X']
        y = track['POSITION_Y']

        # finally, append the current trajectory to the list of trajectories
        trajectories.append(np.array([x, y]).T)
        good_track_ids.append(track_id)
    
    return trajectories

To test this function, we can do the following:

In [ ]:
trajectories = load_spt(mydir, quality = 50)

To finish this part, we can plot *all* the trajectories.

In [ ]:
for trajectory in trajectories :
    plt.plot(trajectory[:,0], trajectory[:,1])
plt.show()

## 1.4 Bulk loading of trajectories
Armed with the function that loads the SPT data of one experiment, we can now proceed to loading *all* the data from a series of experiments. To do that, we will define a "master" directory in which the program will look for sub-directories that match a certain pattern in their names. For each directory, we will invoke the function that loads the data. We must keep the data tidy, so we will define appropriate data structures to keep the data.

In [ ]:
# define the master directory
data_rootdir = '%s/2_DMSO_R5020_Control'%(spt_rootdir)

# we will keep our data in a dictionary
experiments = {}

# iterate over the subdirectories. Notice that the variable `subdir` will contain only
# the name of the subdirectory, not the full path of it.
for subdir in os.listdir(data_rootdir) :
    
    # here we test the name of the directory. We use a regular expression to check
    # whether the name of the subdirectory contains a format "StackN_CellN".
    if re.match('Stack[0-9]+_Cell[0-9]+', subdir) is None :
        continue
    
    # if we get here, we passed the test
    experiments[subdir] = load_spt('%s/%s'%(data_rootdir, subdir), quality=50)

Let's get information on the first few experiments.

In [ ]:
n = 0
for name, trajectories in experiments.items() :
    print(name, len(trajectories))
    n += 1
    if n == 5 : break

We can also get overall statistics on the distribution of the number of tracks and their lengths. First, we create a list that contains the length of the tracks and the number of trajectories for each cell, and then another list that contains the number of points in each trajectory.

In [ ]:
track_lengths = []
n_tracks = []
for name, trajectories in experiments.items() :
    
    # here we use "extend" so that the list will contain a list of number and
    # not a list of lists
    track_lengths.extend([len(t) for t in trajectories])
    
    n_tracks.append(len(trajectories))

In [ ]:
# plot the histogram of number of points in trajectories
plt.hist(track_lengths, bins = 80)
plt.xlabel("Number of points in track")
plt.ylabel("Counts")
plt.show()

In [ ]:
# plot the histogram of the number of trajectories per cell
plt.hist(n_tracks, bins=40)
plt.xlabel("Number of tracks in cell")
plt.ylabel("Counts")
plt.show()

As before, we can create a single function that takes care of getting all the data in bulk.

In [ ]:
def load_experiments(data_rootdir, quality=None) :
    
    # init the output data structure
    experiments = {}

    # iterate over the subdirectories. Notice that the variable `subdir` will contain only
    # the name of the subdirectory, not the full path of it.
    for subdir in os.listdir(data_rootdir) :

        # here we test the name of the directory. We use a regular expression to check
        # whether the name of the subdirectory contains a format "StackN_CellN".
        if re.match('Stack[0-9]+_Cell[0-9]+', subdir) is None :
            continue

        # if we get here, we passed the test
        experiments[subdir] = load_spt('%s/%s'%(data_rootdir, subdir), quality=50)
    return experiments

We have the function to load the data in bulk. Let's load the data, and move on to the data analysis now.

In [ ]:
experiments = load_experiments(data_rootdir, quality=30)

## 2 Angle Analysis

The next step will be to look at the distribution of angles in the trajectories. The definition of the angles will be performed as in the following graph.

![Alt text](angles.svg)

In the figure, the black circles are localizations, each of which have cartesian coordinates $(x, y)$. As a function of time, the particle's position is described by the vectors $\vec{r}(t_1) \equiv \vec{r}_1$, $\vec{r}(t_2) \equiv \vec{r}_2$, ...

To calculate the angles, we need *three* localizations: the first two to define the first *displacement vector* $\vec{r}_{12} = \vec{r}_2 - \vec{r}_1$, and the second and third localizations are used to define the second displacement vector $\vec{r}_{23} = \vec{r}_3 - \vec{r}_2$. Once we have the two vectors $\vec{r}_{12}$ and $\vec{r}_{23}$, we can calculate the angle between the two vectors.

To do this properly, we need to remember the formula for the scalar product of two vectors:
$\vec{v}_1 \cdot \vec{v}_2 = |\vec{v}_1| |\vec{v}_2| \cos(\theta_{12})$. Therefore, the *absolute value* of the angle between the two vectors can be calculated as

$|\theta_{12}| = \arccos \frac{\vec{v}_1 \cdot \vec{v}_2}{|\vec{v}_1| |\vec{v}_2|}$.

The last step is to give a sign to the vector. The trick is to remember that the *vector product* of two vectors in two dimensions points to one direction or the other depending on the relative orientation of the two vectors. For example, if the first vector lays on the $x$-axis ($\vec{v}_1 \equiv (1,0)$), and the second vector bisects the first quadrant ($\vec{v}_2 \equiv (1,1)$), the vector product would be point upwards. In case that the second vector bisects the fourth quadrant ($\vec{v}_2 = (1, -1)$) then the vector product would be point downwards. Since the two vectors are in two dimensions, the vector product only has a $z$ component. So the final formula for the signed angle between the vectors is:

$\theta_{12} = \mathrm{sign}(\vec{v}_1 \times \vec{v}_2) \arccos \frac{\vec{v}_1 \cdot \vec{v}_2}{|\vec{v}_1| |\vec{v}_2|}$.

With these considerations, let's write down the function that calculates the angles. First, let's write a function that calculates the angle between two vectors `v1` and `v2` (in 2D).

In [ ]:
def angle(v1, v2) :
    # calculate the dot product between the two vectors
    d = np.dot(v1, v2)
    if d==1.0 :
        return 0.0
    
    # calculate the z component of the vector product of the two vectors
    v = v1[0]*v2[1] - v1[1]*v2[0]
        
    # calculate the norm of the two vectors
    nv1 = np.linalg.norm(v1)
    nv2 = np.linalg.norm(v2)
    
    # we add a check of sanity: if any of the two difference vectors has
    # zero length, then it means that the particle has not moved, and the angle
    # cannot be defined.
    if nv1 == 0.0 or nv2 == 0.0 :
        return None
    
    # calculate the cosine of the angle
    N = d/(nv1*nv2)
    
    # and finally the signed angle
    return np.sign(v)*np.arccos(N)

The next step is to calculate the angles for a single trajectory. To do that, we need to calculate the _difference between the position vectors_ as explained above. To do that, we use Numpy's `diff` function, which takes the difference between values in an array along a specified dimension.

In [ ]:
def angles_in_trajectory(trajectory) :
    angles = []
    
    # calculate the x and y components of the difference vectors. Note that we
    # only take from the second element of the list onwards, because the first one
    # is always (0, 0)
    delta_X = np.diff(trajectory[:,0], axis=0)[1:]
    delta_Y = np.diff(trajectory[:,1], axis=0)[1:]
    
    # let's now create an array of all the difference vectors. We do the transpose
    # to create a vector that has many rows and 2 columns
    T = np.array([delta_X, delta_Y]).T
    
    # now we do a loop to calculate the angles between the vectors. Note that we start
    # the iteration from the second element of the list.
    for i in range(1, len(T)) :
        a = angle(T[i], T[i-1])
        if a is not None :
            angles.append(a)
    return angles

Let's test this on a single trajectory.

In [ ]:
# calculate the angles in a single trajectory
trajectory = experiments['Stack001_Cell2'][0]
a = angles_in_trajectory(trajectory)

In [ ]:
# make a nice plot of the trajectory and print out the values of the angles
plt.plot(trajectory[:,0], trajectory[:,1], 'o--', markersize=8)
plt.xlabel("X [microns]", fontsize=24)
plt.ylabel("Y [microns]", fontsize=24)
for i in range(trajectory.shape[0]) :
    plt.text(trajectory[i,0], trajectory[i,1], i+1, fontsize=14)
    if i>1 and i<trajectory.shape[0]-1 :
        print("theta_%d = %.4f radians (%.2f degrees)"%(i-1,a[i-2],a[i-2]*180/np.pi))

Note here the sign of the angles, and that for a trajectory with 13 points we have 10 angles.

The next step is to do the calculation of the angles for all the trajectories, for all the cells, for all the experiments. Since we need to do a histogram of the values of the angles, we need to accumulate all the values of the angles. For that, we will do an `extend` of the python lists in an iteration over all the experiments.

In [ ]:
angles = []
for experiment_name, trajectories in experiments.items() :
    for n, trajectory in enumerate(trajectories) :
        a = angles_in_trajectory(trajectory)
        if a is None :
            print(experiment_name, n)
            break
        angles.extend(a)

Now we can do a plot with the distribution of the angles. We will do a nice radial plot, like the pros.

In [ ]:
def plot_angles(angles, bins, title) :
    # the first thing is to create the histogram of the angles, with the specified
    # number of bins
    counts, angle_edges = np.histogram(angles, bins = bins)
    
    # create a vector corresponding to the centers of the bins
    angle_centers = angle_edges[1:] - np.ediff1d(angle_edges)
    
    # init the figure
    fig = plt.figure(figsize = (5,5))
    
    # here we initialize a plot with polar projection
    ax = plt.subplot(111, projection = 'polar')
    
    # plot the histogram
    bars = ax.bar(angle_centers, counts, width = 2*np.pi/bins, edgecolor = 'k')
    
    # finishing touches
    ax.set_title(title, y = 1.1, fontsize = 18)
    ax.set_yticklabels([])
    return fig, ax

In [ ]:
fig, ax = plot_angles(angles, 20, 'Example')
plt.show()

By performing the same analysis for the different experiments, we'll obtain the figures with the angle analysis for all the experiments.

## 3 Diffusion analysis

The diffusion analysis consists in several parts:

1. Analysis of the **displacements**: we first make an histogram of the square displacements of the particles, and then fit a model that takes into account different populations of diffusing particles to extract the diffusion coefficients and size of the populations.
2. Analysis of the **mean square displacement (MSD)** as a function of time, which will allow us to extract the anomalous diffusion parameter, and the mean diffusion coefficient.

### 3.1 Analysis of the displacements

Let's take a population of particles that that diffuses with brownian motion in two dimensions. At any time, a particle starting at the origin of a reference frame will be encountered at time $\Delta t$ in a shell of radius $r$ and thickness $\mathrm{d} r$ with probability given by:

$p(r^2, \Delta t) \mathrm{d} r^2 = \frac{1}{4 D \Delta t} \exp (-\frac{r^2}{4 D \Delta t})$

where $D$ is the diffusion coefficient. One option would therefore be to calculate the distribution of $r^2$ values and fit this formula to it. However, this would depend on the binning chosen to calculate the histogram of the values of $r^2$. A better option is to fit the *cumulative distribution* of the square displacements, which does *not* depend on  the binning. The cumulative distribution is the integral of the above expression, and is given by

$P(r^2, \Delta t)= 1 - \exp (-\frac{r^2}{4 D \Delta t})$

With this expression we can fit a model of diffusion and extract the diffusion coefficient.

Consider then the case that there are $N$ different populations of diffusing particles, where the $i$-th particle type takes up a fraction $f_i$ of the population, and has diffusion coefficient $D_i$. The cumulative distribution of square displacements for the mixed population will be

$P(r^2, \Delta t)= 1 - \sum_{i=1}^m f_i \exp (-\frac{r^2}{4 D_i \Delta t})$

This last formula is the one that we will use for our fits.

The first step now is to extract the distribution of $r^2$ values. Let's illustrate how to do this for a single trajectory.

In [ ]:
# calculate the angles in a single trajectory
trajectory = experiments['Stack001_Cell2'][0]

# step 1: get the displacements. In a first step we perform the difference between
# the positions of the particles using diff, and in the second step we calculate the
# norm of the displacement vectors
r = np.linalg.norm(np.diff(trajectory, axis = 0), axis = 1)

# step 2: get the square displacements, simple enough
r2 = r**2

We can take these pieces of code and put them into a function that will get us the square displacements for a bunch of trajectories:

In [ ]:
r2 = []
for experiment_name, trajectories in experiments.items() :
    for n, trajectory in enumerate(trajectories) :
        r = np.linalg.norm(np.diff(trajectory, axis = 0), axis = 1)
        r2.extend(r**2)

Now we have a list that contains all the square displacement values for all the experiments in our list. Let's plot the histogram of square displacements.

In [ ]:
plt.hist(r2, bins=50)
plt.xlabel(r"r2 [microns 2]", fontsize=24)
plt.ylabel(r"Counts", fontsize=24)

Let's now compute the cumulative distribution. To do that, we will calculate the *cumulative sum* of the histogram and divide by the total number of events. We need to pass these data points then to a fitting function that we will write later. This means that we need to have a vector of values of $x$ and $y$: the $x$ values will be the values of the square displacements, the $y$ values will be the values of the cumulative sum. We need to take care of a few subtleties in this. First, when we use the function `cumsum` of Numpy, this calculates a vector that has the same number of dimensions as the original vector, as in the toy example below.

In [ ]:
x = np.array([9., 2., 3.])
np.cumsum(x)

What we want is actually a vector that also contains the point $(0, 0)$.

Second, when we invoke the `histogram` function of Numpy, it returns two vectors: `h` and `bins`. The former is the value of the counts associated to each bin, whereas the latter corresponds to the *edges* of the bins. Therefore, the number of points of `bins` will be greater by one compared to `h`.

In [ ]:
# define the minimum and maximum value of the square displacement that we want
# to study
m = 0
M = 1.1

# number of points to include in the final histogram
N = 100

# calculate the histogram of square displacements
bins = np.linspace(m, M, N)
h, bins = np.histogram(r2, bins=bins)

The next step is to prepare the $x$ and $y$ vectors for fitting. We will define the right edges of the bins as $x$, and the cumulative sum of the histogram as $y$.

In [ ]:
# prepare x and y for fitting
x = np.zeros(N)
y = np.zeros(N)
x[1:] = bins[1:]
y[1:] = np.cumsum(h)/sum(h)

# and plot
plt.plot(x,y)
plt.xlabel("Square displacement [microns 2]", fontsize=24)
plt.ylabel("Cumulative distribution", fontsize=24)
plt.show()

Now we are ready to do the fitting.

For fitting, we need three ingredients:
1. the data to fit
2. the model that we want to fit
3. a function that takes the data and the model, and returns the optimal parameters

We have the data already. Let's work on the model. To fit the model, we will use the `curve_fit` function from `scipy`. This function accepts as an argument the function to fit, which must be of the form `f(X, *p)`, where `X` contains the data to fit, and `p` the parameters to fit.

Let's write the model that we want to fit so that `curve_fit` will accept it. One must keep in mind that the `X` part of the arguments contains not only the `x` values that the function must calculate, but also any potential constants that the model needs.

In [ ]:
def diffusion_model_1s(X, *p) :
    """
    One species diffusion model
    """
    # let's extract data and contants
    x = X[0]         # data to fit
    dt = X[1]        # Delta t values
    
    # extract the parameters
    D = p[0]
    
    # calculate the result of the model
    y = 1-np.exp(-x/(4*D*dt))
    return y
    
def diffusion_model_2s(X, *p) :
    """
    Two species diffusion model
    """
    # let's extract data and contants
    x = X[0]         # data to fit
    dt = X[1]        # Delta t values
    
    # extract the parameters
    D1 = p[0]
    D2 = p[1]
    f1 = p[2]
    
    # calculate the result of the model
    y = 1-f1*np.exp(-x/(4*D1*dt))-(1-f1)*np.exp(-x/(4*D2*dt))
    return y

Now we can perform the fit.

In [ ]:
# prepare the data to fit
dt = 0.015         # in seconds... = 15ms
X = (x, dt)

# one species diffusion model fit
p0_1s = 5.0      # initial guess for the diffusion coefficient
coeff_1s, var_matrix_1s = curve_fit(diffusion_model_1s, X, y, p0=p0_1s)

# two species diffusion model fit: note that here we use the "bounds" option for
# curve_fit: there is otherwise the chance that the values of f1 will be outside of
# its legitimate bounds [0, 1]
p0_2s = [5.0, 1.0, 0.1]     # initial guess for D1, D2, and f1, respectively
coeff_2s, var_matrix_2s = curve_fit(diffusion_model_2s, X, y, p0=p0_2s,
                                   bounds = (0,
                                             [np.inf, np.inf, 1.0]))

Let's now explore the results of the fit. Let's plot the data together with the best fit for the one-species and the two-species model. We'll also print out the values of the parameters on the figure.

In [ ]:
# init figure
fig = plt.figure(figsize=(6,4))

# plot values of the data and the fitted values
plt.plot(x, y, 'o', markersize=5, label='Data')
plt.plot(x, diffusion_model_1s((x, dt), coeff_1s[0]), label='One species fit')
plt.plot(x, diffusion_model_2s((x, dt), coeff_2s[0],
                                  coeff_2s[1],
                                  coeff_2s[2]), label='Two species fit')

# aesthetic touches
plt.title("Example", fontsize=24)
plt.axhline(y = 1.0, linestyle = '--', color = 'k', linewidth = 0.75)
plt.xlabel("Displacement squared [$\mu$m$^2$]")
plt.ylabel("Cumulative distribution")
plt.xlim(m, M)
plt.xticks(np.arange(0., M, 0.25))
plt.legend(loc='lower right', fontsize = 16)

plt.text(0.25, 0.8, "One species fit: D = %.3f $\mu m^2/s$"%(coeff_1s[0]), fontsize = 14)
plt.text(0.25, 0.45, "Two species fit: D1 = %.3f $\mu m^2/s$\n"
                      "                         D2 = %.3f $\mu m^2/s$\n"
                      "                         f1 = %.3f"%(coeff_2s[0],
                                                            coeff_2s[1],
                                                            coeff_2s[2]), fontsize = 14)
plt.show()

This concludes the part on the diffusion analysis.

### 3.2 Mean square displacement analysis

The next part is to study the mean square displacement (MSD) of the trajectories. The objective is to extract the diffusion coefficient and the anomalous exponent of the diffusion. For a particle that diffuses with anomalous diffusion in 2D the MSD will depend on the temporal interval as follows:

$MSD(t) = 4 D_\alpha t^\alpha$,

where $\alpha$ is the anomalous exponent, that takes the value of $\alpha = 1$ for normal Brownian diffusion. Strictly speaking, the dimensions of the diffusion coefficient are of $[L]^2/[T]$ only if $\alpha = 1$. That's why we indicate the diffusion coefficient as $D_\alpha$.

To obtain the value of $\alpha$, we first need to calculate the MSD, and then fit it to the formula above.

The tricky part is to calculate the MSD. The single particle tracking experiments are performed by taking images every $\Delta t = 15ms$. To get the MSD, we need averages at fixed time intervals. Say that a trajectory contains 10 points. Clearly, there will be 9 points that are at $\Delta t$ time difference, there are 8 points that are at $2 \Delta t$ time difference, 7 at $3\Delta t$, and so on. Only one point in the trajectory is at $9 \Delta t$. This is why the calculation of the MSD will yield accurate estimations at small time intervals, and progressively less and less accurate estimations at larger time intervals.

This also means that for every trajectory that has $T$ time points, we will be able to calculate $T-1$ points for $\Delta t$, $T-2$ for $2 \Delta t$, and so on. At the end, we will perform the averages. We will do this by creating a square matrix of $T-1 \times T-1$ dimensions. The matrix elements will be defined as follows:

$A_{ij} = |\vec{r}(t_i) - \vec{r}(t_i + j \Delta t)|^2$,

that is, the columns contain all the $r^2$ values for a fixed time interval, and the row contain all possible values for that time interval.

To avoid doing cumbersome calculations, we will fill the matrix at the beginning with `NaN`s, and then use Numpy's `nanmean` function to perform the averages: this way, it will automatically perform the average taking into account that only some of the values have been calculated.

With that said, let's look at how we can calculate the values of the MSD for a single trajectory.

In [ ]:
def msd_trajectory(trajectory) :
    # get the length of the trajectory
    T = trajectory.shape[0]

    # initialize a matrix that will allow us to calculate the MSD: the matrix
    # has T-1 rows, corresponding to each time point except the last one (because
    # for the last one we cannot calculate any difference in time); it has then
    # T-1 columns, so that it stores all the possible time intervals from 1 to T
    delta = np.zeros((T-1, T-1))
    delta.fill(np.nan)

    # calculate the MSD for the single trajectory: iterate first on the initial time
    # point, and then on the values of the possible time intervals
    for t0 in range(T-1) :
        for delay in range(1,T-t0) :
            # get value of second time point
            t1 = t0 + delay
            
            # get positions of the particle at the two time points
            pos1 = trajectory[t1,:]
            pos0 = trajectory[t0,:]
            
            # calculate square displacement
            delta[t0, delay-1] = np.sum((pos1-pos0)**2)

    # now we can return the mean of this track
    return np.nanmean(delta, axis=0)

We can test now this function with a single trajectory as an example.

In [ ]:
plt.plot(trajectory[:,0], trajectory[:,1])
msd_trajectory(trajectory)

Finally, we can perform the averages over all the trajectories in an experiment set.

In [ ]:
# this is the function that we will use to calculate the MSD
def msd_t(trajectories) :
    
    # init the variables
    ntrajectories = len(trajectories)
    lengths = [t.shape[0] for t in trajectories]
    max_t = int(max(lengths)) - 1
    
    # We init a matrix that will contain the MSD calculated for each of the
    # tracks. It will have `ntracks` rows and `max_t - 1` columns.
    MSD = np.zeros((ntrajectories, max_t))
    MSD.fill(np.nan)

    # iterate through the all the tracks
    for i, trajectory in enumerate(trajectories) :
        m = msd_trajectory(trajectory)
        MSD[i, 0:len(m)] = m
    
    # return the MSD
    return np.nanmean(MSD, axis=0)

In [ ]:
trajectories = []
for experiment, tracks in experiments.items() :
    trajectories.extend(tracks)
msd = msd_t(trajectories)

Now we have the MSD data. Let's have a quick look.

In [ ]:
plt.loglog(msd)
plt.xlabel(r'$\log[\Delta t]$ [s]', fontsize = 18)
plt.ylabel(r'$\log [MSD (\Delta t)]$', fontsize = 18)

It is quite clear that the data is smooth in the first part, and noisy in the second part. Let's move on to fitting this data. The best way to fit this is to take into consideration that we can transform the MSD law into the following form:

$\log MSD = \log (4D_\alpha) + \alpha \log t$

which means that we can fit a linear law of $\log MSD$ as a function of $\log t$, in which $\log 4D_\alpha$ is the intercept, and $\alpha$ is the slope of the line. Note that this will be possible only in a certain range of values of $t$.

To fit the data, we write a custom linear regression function, that is handy because it returns the errors on the estimates of the parameters, and can be supplied with the confidence interval.

In [ ]:
def linear_regression (x,y,prob) :
    """
    Fit (x,y) to a linear function, using maximum likelihood estimation of the
    confidence intervals on the coefficients, given the user-supplied
    probability *prob*
    """
    n = len(x)
    xy = x*y
    xx = x*x
    # estimates
    xmean = x.mean()
    ymean = y.mean()
    xxmean = xx.mean()
    xymean = xy.mean()
    b1 = (xymean-xmean*ymean) / (xxmean-xmean**2)
    b0 = ymean-b1*xmean
    s2 = 1./n * sum([(y[i] - b0 - b1 * x[i])**2 for i in range(n)])
    #confidence intervals
    alpha = 1 - prob
    c1 = stats.chi2.ppf(alpha/2.,n-2)
    c2 = stats.chi2.ppf(1-alpha/2.,n-2)
    # get results and return
    c = -1 * stats.t.ppf(alpha/2.,n-2)
    bb1 = c * (s2 / ((n-2) * (xxmean - (xmean)**2)))**.5
    bb0 = c * ((s2 / (n-2)) * (1 + (xmean)**2 / (xxmean - xmean**2)))**.5
    return b0,b1,bb0,bb1

Now we can fit.

In [ ]:
# first step: prepare the data to be fitted
# remember that dt was defined earlier
cutoff = 8      # number of points to fit
t = np.arange(1, msd.size+1)*dt
x = np.log(t[:cutoff])
y = np.log(msd[:cutoff])

# now we can fit the model y = a + b * x, where da and db will be the standard
# errors on the estimates of the coefficients of the regression
b, alpha, db, dalpha = linear_regression(x, y, 0.99)

# and finally we transform back to get the values of diffusion coefficient
# with its error
D = np.exp(b)/4.
dD = np.exp(db)/4.

Now we can plot everything together.

In [ ]:
fig = plt.figure()

# t axis
t = dt*np.arange(cutoff)

# the diffusion model: 2D diffusion
yfit = 4 * D * t ** alpha

# plot the data and the fit to the model
plt.loglog(dt*np.arange(msd.shape[0]), msd,
      markersize = 8)
plt.loglog(t, yfit, '--', color = 'k')

plt.text(0.02, 0.01, r'$\alpha$ = %.3f +/-'
       '%.3f\nD = %.3f +/- %.3f $\mu m^2$/s'%(alpha, dalpha, D, dD), fontsize=18)

# finish up with plot style
plt.xlabel(r'$log[\Delta t]$ [s]', fontsize = 18)
plt.ylabel(r'$\log [MSD (\Delta t)]$', fontsize = 18)

plt.show()